To be corrected:
* Taxize
* URL check

Version: 0.1.0

By: Dana Ransby, based on Python version by Michael Oellermann, Kathrin Riemann-Campe

Last updated: 2023-08-28

This notebook aims to help you, to check and resolve some common problems and issues with data tables, before submitting them to a data repository like [PANGAEA](https://www.pangaea.de/submit/).

Check out our [instructions](https://wiki.pangaea.de/wiki/Data_submission) and [data templates](https://wiki.pangaea.de/wiki/Best_practice_manuals_and_templates) for submissions to PANGAEA.



# Import libraries

In [ ]:
library(dplyr)
library(stringr)
library(lubridate)
library(taxize)
library(httr)

# Create practice data tables

- Create random number data file for PANGAEA test submission
- Added mistakes:
  - Date and time in separate columns
  - Latitude and longitude in single cell
  - Different ways of marking missing values
  - Comma separated values
  - Parameter with unit, that needs rescaling
  - () instead of []
  - Unrealisticly high number of decimal points
  - Row with comments
  - Row with aggregated statistics
  - Feature with abbreviations (Threatened status)
  - Species column with misspelled species names
  - Column with NaN only
  - Leading/trailing/double white spaces

In [ ]:
# Function to create fake data
create_fake_df <- function() {
  # Add numeric features
  df <- data.frame(seq(from = 69.2, to = 89, by = 0.1),
                   seq(from = 11, to = 110, by = 0.5),
                   seq(from = 1, to = 100, by = 0.5),
                   runif(length(seq(from = 69.2, to = 89, by = 0.1)))
                   )
  names(df) <- c("latitude (deg)", "lngitude (deg)", "water depth (cm)", "water temperature (degC)" )

  
  # Add object feature
  df$Location <- "Bremerhaven, Germany"
  
  # Add geolocations in degrees and in a single field
  df$`lat/lon` <- "53°32'59.9964''N; 8°34'59.9988''E"
  
  # Create parameter with abbreviations
  threatened_status <- rep(c("LC", "NT", "EN", "EX"), length.out = nrow(df))
  df$`Threatened status` <- threatened_status
  
  # Add species column
  species <- rep(c("Stenella coeruleoalba", "Tursiops truncatus", "Dolphinus delphis"), length.out = nrow(df))
  df$species <- species
  
  # Add row with comments
  df[1, ] <- rep("Comment", ncol(df))
  
  # Add row at the end with "Total sum"
  df[nrow(df) + 1, ] <- rep("Total sum", ncol(df))
  
  # Add empty column
  df$Empty <- NA
  
  ## Date and time manipulations
  # Add date and time columns
  sdate <- as.Date("2010-03-22")   # start date
  edate <- as.Date("2021-04-09")   # end date
  # Generate date and time range
  date_time_range <- seq.POSIXt(from = as.POSIXct(sdate), to = as.POSIXct(edate) - 86400, by = "hour")
  # Sample random date
  df$date <- sample(as.Date(date_time_range), nrow(df), replace = TRUE)
  # Sample random time
  df$time <- format(sample(date_time_range, nrow(df), replace = TRUE), format = "%H:%M:%S")
  
  ## Manipulate values
  # Add leading and trailing white space to values
  df$Location <- sprintf("%3s", df$Location)
  df$Location <- sprintf("%-2s", df$Location)
  # Add double white spaces in between strings
  df$Location[3] <- "Bremerhaven,     Germany"
  
  # Add comma separated value to latitude
  df$`latitude (deg)` <- as.character(df$`latitude (deg)`)
  df$`latitude (deg)`[3] <- "69,4"
  
  # Add missing values in different formats
  df$`water depth (cm)` <- as.character(df$`water depth (cm)`)
  df$`water depth (cm)`[2] <- "-999.9"
  df$`water depth (cm)`[3] <- "n/a"
  
  # Sort columns
  df <- df[, c('date', 'time', 'Location', 'Threatened status', 'species', 'lat/lon', 'latitude (deg)',
               'lngitude (deg)', 'water depth (cm)', 'water temperature (degC)', 'Empty')]
  
  ## Save as csv file
  write.table(df, file = 'test_data.csv', row.names = FALSE, sep = ',', quote = FALSE)
  
  return(df)
}


In [ ]:
# create a test dataset
df <- create_fake_df()
head(df)

# Data Curation Checklist

## Data structure

* One cell = one entry
* Columns = Variable/Parameters
* Rows = Single observation
* Single header row
* No mix of numeric values and strings

### Check data structure

In [ ]:
# Check structure of data table
# head(df, 5)

# Alternative way to see head and tail together
rbind(head(df, 4), tail(df, 4))

### Remove rows that are not observations

Data table should have:

* only single header row
* no rows with aggregated statistics

In [ ]:
# Remove row with comments
df <- df[-1, ]

# Remove row with aggregated statistics
df <- df[-nrow(df), ]

# Concatenate head and tail of the data frame
rbind(head(df, 2), tail(df, 2))

## Data types

* Are data types as expected?

__Numeric data__

* Columns must contain numbers only; exception see [quality flags](https://wiki.pangaea.de/wiki/Quality_flag), DO NOT USE FORMULAS (Excel) - cells with formulas have to be saved as number prior to submission
* Fields without data should be left empty (and NOT filled with '-', 'n/a', 'NaN', -9999 or '*' etc.).

In [ ]:
# Check data types
str(df)

In [ ]:
head(df, 2)

### Remove all columns containing nothing

In [ ]:
# Remove columns containing only NA
df <- df[, colSums(is.na(df)) != nrow(df)]

### Remove ambigious missing values

In [ ]:
# Remove ambiguous missing values
df$`water depth (cm)` <- gsub("-999.9|n/a", "", df$`water depth (cm)`)

### Check for comma separated values

In [ ]:
# Check for comma separated values
# List unique values
unique_values <- unique(df$`latitude (deg)`)
head(unique_values, 10)

In [ ]:
# Replace commas with dots
df$`latitude (deg)` <- gsub(",", ".", df$`latitude (deg)`)

### Convert to numeric

In [ ]:
# Convert specific column to numeric
df$`latitude (deg)` <- as.numeric(df$`latitude (deg)`)

# OR: Convert multiple columns of dataframe to numeric
df[, 7:ncol(df)] <- apply(df[, 7:ncol(df)], 2, as.numeric)

# Check numeric features for text entries
str(df)

### Remove leading and trailing white space

In [ ]:
# Remove all leading and trailing white spaces in columns identified as characters
cols <- sapply(df, is.character)
df[cols] <- lapply(df[cols], str_trim)

# Optional: Remove double white spaces
cols <- sapply(df, is.character)
df[cols] <- lapply(df[cols], function(x) gsub("\\s+", " ", x))
df[cols] <- as.data.frame(lapply(df[cols], function(x) gsub("\\s+", " ", x)))

In [ ]:
# Show the table
rbind(head(df, 5), tail(df, 5))

## Date formatting
Date/Time should be provided in the ISO-format (e.g. 1954-04-07T13:34:11) as coordinated universal time (UTC) <p>
--> for local time it is better to use new "local time" column

### Merge Date and Time and convert to PANGAEA format

In [ ]:
# Join date and time
df$date <- paste0(df$date, df$time)
df$date <- ymd_hms(df$date)

# Convert to pangaea standard time format
df$date <- format(df$date, "%Y-%m-%dT%H:%M:%S")

# Remove redundant time column
df <- subset(df, select = -c(time))

head(df)

### Convert from degrees to decimal format
Note: decimal degree (-65.1234) (S and W are negative, projection WGS84)

#### First separate latitude and longitude into individual columns
Note: Multiple values separated by ';', '-', '±', '()' (ranges, values with errors, uncertainties, or alternative values in brackets) within a single cell should be avoided

In [ ]:
# Split cell by semicolon separator
df$lat <- sapply(strsplit(as.character(df$`lat/lon`), ";"), "[", 1)
df$lon <- sapply(strsplit(as.character(df$`lat/lon`), ";"), "[", 2)

# Remove old lat/lon column
df <- subset(df, select = -c(`lat/lon`))
head(df)

#### then convert from degrees to decimal format

In [ ]:
dms2dec <- function(dms, separators = c("º", "°", "\'", "’", "’’", "\"", "\'\'", "\\?")) {
  
  # version 1.4 (2 Feb 2022) source("https://raw.githubusercontent.com/AMBarbosa/unpackaged/master/dms2dec", encoding = "UTF-8")
  # dms: a vector of latitude or longitude in degrees-minutes-seconds-hemisfere, e.g. 41° 34' 10.956" N (with or without spaces)
  # separators: the characters that are separating degrees, minutes and seconds in 'dms'; mind these are taken in the order in which they appear and not interpreted individually, i.e. 7'3º will be taken as 7 degrees, 3 minutes! input data are assumed to be properly formatted
  
  dms <- as.character(dms)
  dms <- gsub(pattern = " ", replacement = "", x = dms)
  for (s in separators) dms <- gsub(pattern = s, replacement = "_splitHere_", x = dms)
  
  splits <- strsplit(dms, split = "_splitHere_")
  n <- length(dms)
  deg <- min <- sec <- hem <- vector("character", n)
  
  for (i in 1:n) {
    deg[i] <- splits[[i]][1]
    min[i] <- splits[[i]][2]
    
    if (length(splits[[i]]) < 4) {
      hem[i] <- splits[[i]][3]
    } else {
      sec[i] <- splits[[i]][3]
      hem[i] <- splits[[i]][4]
    }
  }
  
  dec <- colSums(rbind(as.numeric(deg), (as.numeric(min) / 60), (as.numeric(sec) / 3600)), na.rm = TRUE)
  sign <- ifelse (hem %in% c("N", "E"), 1, -1)
  hem_miss <- which(is.na(hem))
  if (length(hem_miss) > 0) {
    warning("Hemisphere not specified at position(s) ", hem_miss, ", so the sign of the resulting coordinates may be wrong.")
  }
  dec <- sign * dec
  return(dec)
}  # end dms2dec function


In [ ]:
# Convert latitude and longitude to decimal format
df$lat<- dms2dec(df$lat)
df$lon<- dms2dec(df$lon)
head(df)

## Spelling

### Spell out abbreviations

In [ ]:
# What are the abbreviations
unique_abbreviations <- unique(df$`Threatened status`)

# Create list with abbreviations
abbreviated <- c("LC", "NT", "EN", "EX")

# Create list with full names
spelled_out <- c("Least Concern", "Near Threatened", "Endangered", "Extinct in the Wild")

# Replace the values in 'Threatened status' column with the named vectors
df$`Threatened status` <- spelled_out[match(df$`Threatened status`, abbreviated)]
head(df)

### Correct species names

In [ ]:
# Create data frame with unique species names
species <- data.frame(name = unique(df$species))

# Save as tab-separated values (tsv) file for upload to WoRMS or ITIS
write.table(species, file = "Species.txt", sep = "\t", row.names = FALSE, quote = FALSE)

# Print the species data frame
print(species)

--> Use the species.txt file to check species spelling using the taxon match tools of:
- [WoRMS](https://www.marinespecies.org/aphia.php?p=match) 
- [ITIS](https://www.itis.gov/taxmatch.html)

After this you can also add columns for the AphiaID (WoRMS) or TSN numbers (ITIS)
For species spelling conventions read our [PANGAEA wiki](https://wiki.pangaea.de/wiki/Taxon)

#### taxize
Alternatively, use the R package [taxize](https://cran.r-project.org/web/packages/taxize/index.html)

In [ ]:
# Using package taxize
matched_species <- tax_name(species$name, get = "species" )
print(matched_species)

##### Correct individual species names

In [ ]:
# Correct misspelled species names
df$species <- gsub("Dolphinus delphis", "Delphinus delphis", df$species, ignore.case = TRUE)
head(df)

## Parameter (header) naming

* Use clear and descriptive parameter names, no cryptic abbreviations. Non experts should be able to understand the meaning (e.g. "Water content, sediment [%]" instead of "WCS")
* All parameters in English only
* Parameters are always accompanied by a unit
* Use SI units written in square brackets following the parameter e.g. "Ground heat, flux [W/m**2]"
* Find suitable standard parameters names by:
  * Looking at similar datasets published at [PANGAEA](https://pangaea.de/)
  * Finding matching parameters in the [PANGAEA parameter database](https://www.pangaea.de/lists/parameter/all-byname)
* Look out for PANGAEA's special [geocodes](https://wiki.pangaea.de/wiki/Geocode) and their formatting

### Download complete PANGAEA parameter list

In [ ]:
params <-read.csv("https://www.pangaea.de/lists/parameter/all-byname", sep = "\t")
# Check table size
cat(paste("There are currently", nrow(params), "parameters available in PANGAEA\n"))

In [ ]:
# Display the first few rows of the data frame
head(params)

### Lookup similar parameters in the existing list of PANGAEA parameters
Note: If you cant find a suitable parameter we can create a new one for you

#### Find parameters containing your parameter name
Note: Also check for a matching unit for your parameter

In [ ]:
# Find your parameter (example: "Latitude") using partial string matching (case-insensitive)
params[grepl("Latitude", params$Parameter, ignore.case = TRUE), ]

In [ ]:
# Find similar (not exact matches) PANGAEA parameters for your own parameter. This search is less sensitive to spelling mistakes
params[agrepl("latitude", params$Parameter, ignore.case = TRUE), ]

### Rename parameters

In [ ]:
# Rename parameters
colnames(df) <- c('DATE/TIME []', 'Location []', 'IUCN Red List status []', 'Species []', 'LATITUDE []', 'LONGITUDE []', 
                  'DEPTH, water [m]', 'Temperature, water [°C]', 'Latitude 2 []', 'Longitude 2 []')

head(df, 2)

## Conversions
### Convert units
PANGAEA aims to reduce redundancy of units by converting units to one scale (if possible). During the lookup of parameters you can already see which unit scale is used in PANGAEA.

In [ ]:
# DEPTH, water was initially in "cm", but in PANGAEA "m" is the most common unit used for this parameter
df$`DEPTH, water [m]` <- df$`DEPTH, water [m]` / 100
head(df, 5)

## Decimals

In [ ]:
# Temperature hast too many decimals, depending on the device used, 3 decimals might be more suitable
df$`Temperature, water [°C]` <- round (df$`Temperature, water [°C]`, 3)
head(df, 3)

## URLs

In [ ]:
df$'Uniform resource locator/link to reference []'<- "https://doi.org/10.1594/PANGAEA.945749"
head(df, 5)

In [ ]:
# Function to check if URL is valid
check_url <- function(url) {
  response <- try(GET(url))
  return(response$status_code == 200)
}

In [ ]:
# Apply the function to the unique URLs in the column to check if they are any invalid URLs
unique_URL <- data.frame('URL' = unique(df$`Uniform resource locator/link to reference []`))
unique_URL$`url check` <- sapply(unique_URL$URL, check_url)
head(unique_URL, 5)

## 3.8. Event
An [EVENT](https://wiki.pangaea.de/wiki/Event) refers to the sampling event at which the data was obtained, such as a [station](https://www.pangaea.de/expeditions/) during a research expedition.

In [ ]:
df$'Event []' <- "PS132/1-2"
head(df, 2)

# 4. Save curated data
* as Excel or tab-delimited text files
* Important: PANGAEA only accepts UTF-8 encoding to avoid unicode errors

In [ ]:
# Save curated data with a new column sequence
write.table(df[,c(12, 1:11)], file = "Curated_data.txt", sep = "\t", quote = FALSE, row.names = FALSE, fileEncoding = "UTF-8")

# 5. Submit to PANGAEA
Congratulations! You finished curating your data, which was an important step forward to make your data more accessible, understandable und useable for future users.
Now you are ready to submit your data file(s) to [PANGAEA](https://www.pangaea.de/submit/).

